In [3]:
# 获得数据
#了解数据每一项的含义
# 缺失数据怎么处理
# 文本到数字的映射
# 分片训练集和测试集
# 属性统一尺度
# 选择重要的属性
# 通过随机森林选择属性

import pandas as pd
import numpy as np

In [63]:
df = pd.read_csv('d:/project/data/block_min.txt', delimiter='\t')
df.head()

,site,qt,uptime,wrapperid,price,baseprice,addprice,tax,uname,pinfo,...,deptime,fromdate,retdate,rprice,rbaseprice,rtax,coreblockdesc,qunarglobal,biddingcase,pricekey
0,cpb.trade.qunar.com,9eb9c41b6a31A,2017-9-26 00:00:00,ttsgjd10132,3010.0,3000,10,20,NaN,21#0,...,09:15,2017-12-20,NaN,3010.0,3010.0,20.0,NaN,NaN,NaN,XCJUCJJ2017-12-20#200#ITTS#cpb.trade.qunar.com...
1,cpb.trade.qunar.com,9fb9c41c8a94A,2017-9-26 00:00:00,ttsgjd10132,3046.0,3000,46,20,NaN,21#0,...,09:15,2017-12-20,NaN,3046.0,3046.0,20.0,NaN,NaN,NaN,XCJUCJJ2017-12-20#200#ITTS#cpb.trade.qunar.com...
2,mzi.trade.qunar.com,m_b8b588b36e07440582825f1ece4bf0dc,2017-9-26 00:00:00,ttsgjs00198,3290.0,3570,21,3559,yyfczpx1195,"70#290#TEK_855717320_QLXCPCN1_3RTN,TEK_8557173...",...,00:05/10:05~16:20/02:50,2017-10-17,2017-10-25,3290.0,3570.0,3559.0,NaN,NaN,NaN,XSHAMRU2017-10-17~MRUSHA2017-10-25#200#ITTS#mz...
3,qlh.th.qunar.com,m_2bd5bf0cd59f4c20b90268a8ebb48e17,2017-9-26 00:00:00,ttsgjdt2409,19.0,60,-8,427,YMSH,70#330#TSC_847989843_UOW16T_5OWN@ETERM@QFARE,...,01:00,2017-10-09,NaN,19.0,60.0,427.0,NaN,NaN,NaN,"XKMGBKK2017-10-09#200#ITTS#qlh.th.qunar.com#{""..."
4,yyi.trade.qunar.com,m_bbe3936877854ce18711587e422cce94,2017-9-26 00:00:01,ttsgjs00054,922.0,900,152,1960,nfzm1545,"70#92954#T3U_26300919185492233_VL1MCL_20RTN,T3...",...,10:20/19:20~00:55/15:30,2017-10-17,2017-10-25,922.0,900.0,1960.0,NaN,NaN,NaN,XSZXLAX2017-10-17~LAXSZX2017-10-25#200#ITTS#yy...


In [8]:
df.columns

Index(['site', 'qt', 'uptime', 'wrapperid', 'price', 'baseprice', 'addprice',
       'tax', 'uname', 'pinfo', 'pricetag', 'flighttype', 'ispackage',
       'source', 'fromcity', 'tocity', 'flightnum', 'cabin', 'depdate',
       'deptime', 'fromdate', 'retdate', 'rprice', 'rbaseprice', 'rtax',
       'coreblockdesc', 'qunarglobal', 'biddingcase', 'pricekey'],
      dtype='object')

In [65]:
df['price_change'] = df['price'] - df['rprice']
df['tax_change'] = df['tax'] - df['rtax']

In [69]:
df['ok'] = 0
df['ok'][df['coreblockdesc'].notnull()] = 1

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [72]:
df['ok'==1]

KeyError: False

In [55]:
def get_carrier(key):
    if type(key) != str:
        return ''
    k1 = key.split('~')
    ca = ''
    for i, v in enumerate(k1):
        if i>0:
            ca+='~'
        k2 = v.split('/')
        for j, m in enumerate(k2):
            if j > 0:
                ca+='/'
            ca += m[0:2]
    return ca

print(get_carrier('LJ201'))
print(get_carrier('EK303/EK703~EK702/EK302'))
print(get_carrier('EK703~EK702'))
print(get_carrier('3U8702/3U8699'))

LJ
EK/EK~EK/EK
EK~EK
3U/3U


In [56]:
df['carrier'] = df['flightnum'].apply(get_carrier)

In [58]:
def split_pinfo(pinfo):
    if type(pinfo) != str:
        return pinfo
    return pinfo.split('#')[0]
df['pinfo']=df['pinfo'].apply(split_pinfo)

In [59]:
site_mapping = {label:idx for idx,label in enumerate(np.unique(df['site']))}
pinfo_mapping = {label:idx for idx,label in enumerate(np.unique(df['pinfo']))}
fromcity_mapping = {label:idx for idx,label in enumerate(np.unique(df['fromcity']))}
tocity_mapping = {label:idx for idx,label in enumerate(np.unique(df['tocity']))}
flightnum_mapping = {label:idx for idx,label in enumerate(np.unique(df['flightnum']))}
cabin_mapping = {label:idx for idx,label in enumerate(np.unique(df['cabin']))}
carrier_mapping = {label:idx for idx,label in enumerate(np.unique(df['carrier']))}

df['site'] = df['site'].map(site_mapping)
df['pinfo'] = df['pinfo'].map(pinfo_mapping)
df['fromcity'] = df['fromcity'].map(fromcity_mapping)
df['tocity'] = df['tocity'].map(tocity_mapping)
df['carrier'] = df['carrier'].map(carrier_mapping)
df['cabin'] = df['cabin'].map(cabin_mapping)
df['flightnum'] = df['flightnum'].map(flightnum_mapping)
df.head()

TypeError: unorderable types: int() > str()

In [60]:
site_mapping = {label:idx for idx,label in enumerate(np.unique(df['site']))}

TypeError: unorderable types: int() > str()

In [70]:
df['site'].tail()

9994       qnp.th.qunar.com
9995       qnp.th.qunar.com
9996    qhx.trade.qunar.com
9997    gjp.trade.qunar.com
9998    gzz.trade.qunar.com
Name: site, dtype: object